In [14]:
import os
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import gensim.utils
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import itertools
import pandas as pd

# filter out warnings
import warnings

[nltk_data] Downloading package punkt to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
stop = stopwords.words("english")

[nltk_data] Downloading package stopwords to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Helper method
#function to normalize description tokenize remove stopwords
def norm(desc):
    desc = str(desc)
    desc = re.sub('[^a-zA-Z]',' ',desc)
    desc = desc.lower()
    desc = re.sub(pattern='\s+', repl=' ', string=desc)
    desc = nltk.word_tokenize(desc)
    return desc


# Get  the nouns / proper nouns / adjectives
def postagwords(tagged_stanzas):
  chunkGram = r"""NP: {<NN>+<PUNCT>+}"""
  chunkGram = r"""chunk: {<JJ>+}"""
  chunkGram = r"""chunk: {<NN>+<NN>+<NN>+}"""
  #Chunk 3: Sequence of Nouns
  chunkGram = r"""chunk: {<NN|NNP|NNS|NNPS>+}"""
  chunkGram = r"""NP: {<NN|NNP|NNS|NNPS>+}"""
  chunkParser = nltk.RegexpParser(chunkGram)
  nouns = []
  finalpos= []

  for i in tagged_stanzas:
    tree1 = chunkParser.parse(i)
    nouns = [(word) for (word, pos) in i if pos == 'NN' or pos == 'NNS' or pos=='NNP' or pos=='JJ']
    finalpos.append(nouns)

  flat_list = list(itertools.chain(*finalpos))
  return flat_list



#Skills Frequency
def most_frequent_skills(flat_list):
  tag_fd = nltk.FreqDist(flat_list)
  countsskills = pd.DataFrame(tag_fd.items(), columns=['word', 'frequency'])
  # print('before')
  # print(countsskills)
  countsskills =countsskills[countsskills['word'].isin(['statistics', 'machine',  'r', 'python', 'nlp',  'neural','forecasting','tableau','power','bi',
                        'software','programming'  'cloud' , 'aws','sas','regression','predictive', 'modeling','clustering' ,'database','databases',
                        'datawarehouse','excel','pytorch','gcp','ETL','business','visualization','macros','operations','documentation','azure','ops','microsoft','powerpoint'
                        'communication','interpersonal','storytelling','marketing','terradata','splunk','ruby','spark','cassandra','mongodb','scala','api','segmentation','targeting',
                        'reporting','quality','a/b','html','css','sales','visio','logistics'
                        'project','access','qa','quality','agile','scrum','jira'])]
  countsskills.sort_values(by=['frequency'],ascending=False)
  # print('after')
  # print(countsskills)
  return countsskills[:30]

In [17]:
job_df=pd.read_csv('../data/job_posting.csv')
print(len(job_df))

15886


In [18]:
# preview
job_df.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,85008768,NaN,Licensed Insurance Agent,While many industries were hurt by the last fe...,52000.0,NaN,45760.0,YEARLY,Full-time,"Chico, CA",...,1708301126000,NaN,NaN,NaN,1692749193000,NaN,1,FULL_TIME,USD,BASE_SALARY
1,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,NaN,NaN,NaN,NaN,Full-time,"Santa Clarita, CA",...,1695426553000,NaN,NaN,NaN,1692834553000,NaN,0,FULL_TIME,NaN,NaN
2,133196985,1089558.0,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,NaN,NaN,NaN,NaN,Contract,"New York, NY",...,1695321872000,NaN,NaN,NaN,1692729872000,NaN,0,CONTRACT,NaN,NaN
3,381055942,96654609.0,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,NaN,NaN,NaN,NaN,Full-time,"Forney, TX",...,1695424007000,NaN,NaN,NaN,1692832007000,NaN,0,FULL_TIME,NaN,NaN
4,529257371,1244539.0,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,NaN,NaN,NaN,NaN,Full-time,"New York, NY",...,1708419602000,NaN,NaN,NaN,1692867714000,NaN,1,FULL_TIME,NaN,NaN


In [19]:
# keep title and description column only
job_df = job_df[['title', 'description', 'views']]

In [20]:
#running function on description column
job_df_Desc=job_df['description'].apply(lambda x:norm(x))
job_df_Desc

0        [while, many, industries, were, hurt, by, the,...
1        [are, you, a, dynamic, and, creative, marketin...
2        [join, us, as, a, model, risk, auditor, showca...
3        [business, managerfirst, baptist, church, forn...
4        [you, could, be, one, of, the, magic, makers, ...
                               ...                        
15881    [location, west, columbia, sc, us, job, type, ...
15882    [job, title, unit, secretary, department, nurs...
15883    [job, title, radiology, aide, perdiem, departm...
15884    [grade, job, type, officer, of, administration...
15885    [nexion, health, management, affiliates, opera...
Name: description, Length: 15886, dtype: object

In [21]:
# for empty views cell replace with 0
job_df['views'].fillna(0, inplace=True)

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
#removing stopwords from description
stop= nltk.corpus.stopwords.words('english')
new=['job','note','sexual','orientation','equal','opportunity','gender','plus','years','scientist','analyst','based','data','knowledge','discrimination','experience','description','summary','information','etc','eg','layers']
stop.extend(new)
job_df_clean=job_df_Desc.apply(lambda x:[item for item in x if item not in set(stop)])

In [24]:
job_df.loc[:,'tokenized_Description'] = job_df_clean
job_df.loc[:,'tokenized_Description']

0        [many, industries, hurt, last, people, still, ...
1        [dynamic, creative, marketing, professional, l...
2        [join, us, model, risk, auditor, showcase, fin...
3        [business, managerfirst, baptist, church, forn...
4        [could, one, magic, makers, ken, fulk, inc, se...
                               ...                        
15881    [location, west, columbia, sc, us, type, baker...
15882    [title, unit, secretary, department, nursing, ...
15883    [title, radiology, aide, perdiem, department, ...
15884    [grade, type, officer, administrationbargainin...
15885    [nexion, health, management, affiliates, opera...
Name: tokenized_Description, Length: 15886, dtype: object

In [25]:
df = job_df
df["description"] = df["description"].fillna("")
df["tokenized_Description"]

0        [many, industries, hurt, last, people, still, ...
1        [dynamic, creative, marketing, professional, l...
2        [join, us, model, risk, auditor, showcase, fin...
3        [business, managerfirst, baptist, church, forn...
4        [could, one, magic, makers, ken, fulk, inc, se...
                               ...                        
15881    [location, west, columbia, sc, us, type, baker...
15882    [title, unit, secretary, department, nursing, ...
15883    [title, radiology, aide, perdiem, department, ...
15884    [grade, type, officer, administrationbargainin...
15885    [nexion, health, management, affiliates, opera...
Name: tokenized_Description, Length: 15886, dtype: object

In [26]:
import regex as re
df['tokenized_desc_join'] = [' '.join(map(str, l)) for l in df['tokenized_Description']]
df.head()

,title,description,views,tokenized_Description,tokenized_desc_join
0,Licensed Insurance Agent,While many industries were hurt by the last fe...,5.0,"[many, industries, hurt, last, people, still, ...",many industries hurt last people still need in...
1,Sales Manager,Are you a dynamic and creative marketing profe...,0.0,"[dynamic, creative, marketing, professional, l...",dynamic creative marketing professional lookin...
2,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,17.0,"[join, us, model, risk, auditor, showcase, fin...",join us model risk auditor showcase financial ...
3,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,0.0,"[business, managerfirst, baptist, church, forn...",business managerfirst baptist church forneyfor...
4,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,2.0,"[could, one, magic, makers, ken, fulk, inc, se...",could one magic makers ken fulk inc seeking st...


In [27]:
# save the data for faster loading
df.to_csv('../data/job_posting_clean.csv', index=False)

In [28]:
model = Word2Vec(df['tokenized_Description'] ,
                               window=10,
                               min_count=10,
                               workers=5)
model.train(df['tokenized_Description'] ,total_examples=700,epochs=model.epochs)
model.build_vocab(df['tokenized_Description'],progress_per=100,update=True)

In [29]:
# data science specific skills
model.wv.most_similar(['statistics', 'machine', 'r', 'python',
                      'business', 'software', 'aws','sas','regression','predictive', 'modeling' ],topn=25)

[('visualization', 0.8080564737319946),
 ('modelling', 0.7891585826873779),
 ('algorithms', 0.7854996919631958),
 ('scripting', 0.7810526490211487),
 ('matlab', 0.775390088558197),
 ('ml', 0.7590502500534058),
 ('etl', 0.754277229309082),
 ('query', 0.7409961819648743),
 ('statistical', 0.7326993942260742),
 ('nlp', 0.7283128499984741),
 ('sql', 0.7242838740348816),
 ('powerbi', 0.7142824530601501),
 ('nosql', 0.7142578363418579),
 ('hadoop', 0.7115639448165894),
 ('shell', 0.7107833027839661),
 ('spark', 0.7104853391647339),
 ('debugging', 0.7102362513542175),
 ('frameworks', 0.7091355323791504),
 ('simulation', 0.7064521908760071),
 ('etcexperience', 0.7064029574394226),
 ('perl', 0.7059974670410156),
 ('relational', 0.7048850059509277),
 ('informatica', 0.701752245426178),
 ('gitlab', 0.6931585073471069),
 ('tableau', 0.6926621198654175)]

In [30]:
temp = model.wv.most_similar(['finance', 'accounting', 'banking', 'business', 'management'],topn=10,)
temp

[('treasury', 0.6755032539367676),
 ('transactional', 0.6251596808433533),
 ('valuation', 0.5928547978401184),
 ('budgeting', 0.5658106207847595),
 ('fp', 0.5550159811973572),
 ('financial', 0.554163932800293),
 ('mba', 0.5476995706558228),
 ('ap', 0.5454939603805542),
 ('erp', 0.5415525436401367),
 ('traders', 0.5345434546470642)]

In [31]:
temp[1][0]

'transactional'